# Operations problems with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports the construction and solution of optimal power system
scheduling problems (Operations Problems). Operations problems form the fundamental
building blocks for [sequential simulations](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/02_sequential_simulations.ipynb).
This example shows how to specify and customize a the mathematics that will be applied to the data with
an `OperationsProblemTemplate`, build and execute an `OperationsProblem`, and access the results.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using PowerSystemCaseBuilder

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using Cbc #solver

### Data
This data depends upon the [RTS-GMLC](https://github.com/gridmod/rts-gmlc) dataset. Let's
use [PowerSystemCaseBuilder.jl](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/10_PowerSystemCaseBuilder.ipynb) to download and build a `System`.

In [5]:
sys = build_system(PSITestSystems, "modified_RTS_GMLC_DA_sys")

[ Info: Loaded time series from storage file existing=modified_RTS_GMLC_DA_sys_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_oxxqPU
┌ Warning: Rate 500.0 MW for C31-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/N2l8o/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 175.0 MW for B8 is larger than the max expected in the range of (min = 47.0, max = 52.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/N2l8o/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for B26 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/N2l8o/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for A32-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/N2l8o/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for C

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HVDCLine,DCBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
7,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,19
8,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
9,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,21


## Define a problem specification with an `OperationsProblemTemplate`
You can create an empty template with:

In [6]:
template_uc = OperationsProblemTemplate()


Operations Problem Specification
Transmission: CopperPlatePowerModel
Devices Models: 

Branches Models: 

Services Models:



Now, you can add a `DeviceModel` for each device type to create an assignment between PowerSystems device types
and the subtypes of `AbstractDeviceFormulation`. PowerSimulations has a variety of different
`AbstractDeviceFormulation` subtypes that can be applied to different PowerSystems device types,
each dispatching to different methods for populating optimization problem objectives, variables,
and constraints.

In [7]:
print_tree(PSI.AbstractDeviceFormulation)

AbstractDeviceFormulation
├─ FixedOutput
├─ AbstractBranchFormulation
│  ├─ AbstractDCLineFormulation
│  │  ├─ HVDCDispatch
│  │  ├─ HVDCLossless
│  │  └─ HVDCUnbounded
│  ├─ StaticBranch
│  ├─ StaticBranchBounds
│  └─ StaticBranchUnbounded
├─ AbstractHydroFormulation
│  ├─ AbstractHydroDispatchFormulation
│  │  ├─ HydroDispatchRunOfRiver
│  │  └─ AbstractHydroReservoirFormulation
│  │     ├─ HydroDispatchPumpedStorage
│  │     ├─ HydroDispatchPumpedStoragewReservation
│  │     ├─ HydroDispatchReservoirBudget
│  │     └─ HydroDispatchReservoirStorage
│  └─ AbstractHydroUnitCommitment
│     ├─ HydroCommitmentReservoirBudget
│     ├─ HydroCommitmentReservoirStorage
│     └─ HydroCommitmentRunOfRiver
├─ AbstractLoadFormulation
│  ├─ AbstractControllablePowerLoadFormulation
│  │  ├─ DispatchablePowerLoad
│  │  └─ InterruptiblePowerLoad
│  └─ StaticPowerLoad
├─ AbstractRegulationFormulation
│  ├─ DeviceLimitedRegulation
│  └─ ReserveLimitedRegulation
├─ AbstractRenewableFormulation
│  └─ Ab

### Branch Formulations
Here is an example of relatively standard branch formulations. Other formulations allow
for selective enforcement of transmission limits and greater control on transformer settings.

In [8]:
set_device_model!(template_uc, Line, StaticBranch)
set_device_model!(template_uc, Transformer2W, StaticBranch)
set_device_model!(template_uc, TapTransformer, StaticBranch)

### Injection Device Formulations
Here we define template entries for all devices that inject or withdraw power on the
network. For each device type, we can define a distinct `AbstractDeviceFormulation`. In
this case, we're defining a basic unit commitment model for thermal generators,
curtailable renewable generators, and fixed dispatch (net-load reduction) formulations
for `HydroDispatch` and `RenewableFix` devices.

In [9]:
set_device_model!(template_uc, ThermalStandard, ThermalStandardUnitCommitment)
set_device_model!(template_uc, RenewableDispatch, RenewableFullDispatch)
set_device_model!(template_uc, PowerLoad, StaticPowerLoad)
set_device_model!(template_uc, HydroDispatch, FixedOutput)
set_device_model!(template_uc, HydroEnergyReservoir, HydroDispatchRunOfRiver)
set_device_model!(template_uc, RenewableFix, FixedOutput)

### Service Formulations
We have two `VariableReserve` types, parameterized by their direction. So, similar to
creating `DeviceModel`s, we can create `ServiceModel`s. The primary difference being
that `DeviceModel` objects define how constraints get created, while `ServiceModel` objects
define how constraints get modified.

In [10]:
set_service_model!(template_uc, VariableReserve{ReserveUp}, RangeReserve)
set_service_model!(template_uc, VariableReserve{ReserveDown}, RangeReserve)

### Network Formulations
Finally, we can define the transmission network specification that we'd like to model. For simplicity, we'll
choose a copper plate formulation. But there are dozens of specifications available through
an integration with [PowerModels.jl](https://github.com/lanl-ansi/powermodels.jl). *Note that
many formulations will require appropriate data and may be computationally intractable*

In [11]:
set_transmission_model!(template_uc, CopperPlatePowerModel)

## `OperationsProblem`
Now that we have a `System` and an `OperationsProblemTemplate`, we can put the two together
to create an `OperationsProblem` that we solve.

### Optimizer
It's most convenient to define an optimizer instance upfront and pass it into the
`OperationsProblem` constructor. For this example, we can use the free Cbc solver with a
relatively relaxed MIP gap (`ratioGap`) setting to improve speed.

In [12]:
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Build an `OperationsProblem`
The construction of an `OperationsProblem` essentially applies an `OperationsProblemTemplate`
to `System` data to create a JuMP model.

In [13]:
op_problem = OperationsProblem(template_uc, sys; optimizer = solver, horizon = 24)

build!(op_problem, output_dir = mktempdir())

BuildStatus.BUILT = 0

The principal component of the `OperationsProblem` is the JuMP model. For small problems,
you can inspect it by simply printing it to the screen:
```julia
op_problem.internal.optimization_container.JuMPmodel
```

For anything of reasonable size, that will be unmanageable. But you can print to a file:
```julia
f = open("testmodel.txt","w"); print(f,op_problem.internal.optimization_container.JuMPmodel); close(f)
```

In addition to the JuMP model, an `OperationsProblem` keeps track of a bunch of metadata
about the problem and some references to pretty names for constraints and variables.
All of these details are contained within the `optimization_container` field.

In [14]:
print_struct(typeof(op_problem.internal.optimization_container))

mutable struct PowerSimulations.OptimizationContainer
    JuMPmodel::Union{Nothing, JuMP.AbstractModel}
    time_steps::UnitRange{Int64}
    resolution::TimePeriod
    settings::PowerSimulations.Settings
    settings_copy::PowerSimulations.Settings
    variables::Dict{Symbol, AbstractArray}
    constraints::Dict{Symbol, AbstractArray}
    cost_function::JuMP.AbstractJuMPScalar
    expressions::Dict{Symbol, JuMP.Containers.DenseAxisArray}
    parameters::Union{Nothing, Dict{Symbol, PowerSimulations.ParameterContainer}}
    initial_conditions::Union{Nothing, PowerSimulations.InitialConditions}
    pm::Union{Nothing, PowerModels.AbstractPowerModel}
    base_power::Float64
    solve_timed_log::Dict{Symbol, Any}
end


### Solve an `OperationsProblem`

In [15]:
solve!(op_problem)

RunStatus.SUCCESSFUL = 0

## Results Inspection
PowerSimulations collects the `OperationsProblem` results into a struct:

In [16]:
print_struct(PSI.ProblemResults)

 struct ProblemResults
    base_power::Float64
    timestamps::StepRange{DateTime, Millisecond}
    system::Union{Nothing, System}
    variable_values::Dict{Symbol, DataFrames.DataFrame}
    dual_values::Dict{Symbol, DataFrames.DataFrame}
    parameter_values::Dict{Symbol, DataFrames.DataFrame}
    optimizer_stats::PowerSimulations.OptimizerStats
    output_dir::String
end


In [17]:
res = ProblemResults(op_problem);

### Optimizer Stats
The optimizer summary is included

In [18]:
get_optimizer_stats(res)

PowerSimulations.OptimizerStats(0, 0, 1.319522463200657e6, 1, 1, 0, 7.049685955047607, 15.034690443, 1.276909113e9, 1.020961253)

### Objective Function Value

In [19]:
get_objective_value(res)

1.319522463200657e6

### Variable Values
The solution value data frames for variables can be accessed by:

In [20]:
variable_values = get_variables(res)

Dict{Symbol, DataFrames.DataFrame} with 11 entries:
  :P__ThermalStandard       => 24×65 DataFrame…
  :P__RenewableDispatch     => 24×31 DataFrame…
  :Reg_Down__VariableReser… => 24×91 DataFrame…
  :P__HydroEnergyReservoir  => 24×20 DataFrame…
  :Reg_Up__VariableReserve… => 24×91 DataFrame…
  :Spin_Up_R2__VariableRes… => 24×22 DataFrame…
  :On__ThermalStandard      => 24×65 DataFrame…
  :start__ThermalStandard   => 24×65 DataFrame…
  :stop__ThermalStandard    => 24×65 DataFrame…
  :Spin_Up_R1__VariableRes… => 24×31 DataFrame…
  :Spin_Up_R3__VariableRes… => 24×40 DataFrame…

## Plotting
Take a look at the examples in [the plotting folder.](../../notebook/3_PowerSimulations_examples/Plotting)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*